# BAM to GedMatch Batch Conversion (Google Colab)

**Convert Bodzia cemetery BAM files to GedMatch-compatible 23andMe format.**

## Priority Samples (Grave E864/I)

| Sample | Identity | Drive Link |
|--------|----------|------------|
| **VK155** | Bodzia female (mtDNA H1c) | [Download](https://drive.google.com/file/d/1H03qH351o_RemyhZeSf2euICP-OE4Spw/view) |
| **VK157** | Bodzia elite male (Y-DNA I-S2077) | [Download](https://drive.google.com/file/d/1qK86VzMrAA_pzf1okvkjCgW5q2_XPAZS/view) |

**Runtime:** ~15-30 min per sample after reference download (~45 min one-time)

**Source folder:** [Google Drive](https://drive.google.com/drive/folders/1X7XTgWEF7h95QfJbv4tOo493O68Mo9UH)

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 2: Install bioinformatics tools
!apt-get update -qq
!apt-get install -qq bcftools samtools tabix plink1.9
!ln -sf /usr/bin/plink1.9 /usr/bin/plink

print("Tools installed:")
!bcftools --version | head -1
!samtools --version | head -1
!plink --version | head -1

In [ ]:
# Step 3: Download reference genome (one-time, ~10 min)
import os

REF_GENOME = "/content/reference/human_g1k_v37.fasta"

if not os.path.exists(REF_GENOME):
    print("Downloading GRCh37 reference genome (~3GB)...")
    !mkdir -p /content/reference
    !wget -q --show-progress ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz -O /content/reference/human_g1k_v37.fasta.gz
    print("Decompressing...")
    !gunzip -f /content/reference/human_g1k_v37.fasta.gz
    print("Indexing...")
    !samtools faidx /content/reference/human_g1k_v37.fasta
    print("✓ Reference genome ready!")
else:
    print("✓ Reference genome already present")

In [ ]:
# Step 4: Download dbSNP (one-time, ~30 min)
DBSNP = "/content/dbsnp/All_20180423.vcf.gz"

if not os.path.exists(DBSNP):
    print("Downloading dbSNP b151 (~10GB)...")
    !mkdir -p /content/dbsnp
    !wget -q --show-progress ftp://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/VCF/GATK/All_20180423.vcf.gz -O /content/dbsnp/All_20180423.vcf.gz
    !wget -q ftp://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/VCF/GATK/All_20180423.vcf.gz.tbi -O /content/dbsnp/All_20180423.vcf.gz.tbi
    print("✓ dbSNP ready!")
else:
    print("✓ dbSNP already present")

In [ ]:
# Step 5: Configure BAM files
import os
import subprocess

# Output locations
DRIVE_OUTPUT = "/content/drive/MyDrive/GedMatch_Files"
OUTPUT_DIR = "/content/output"
BAM_DIR = "/content/bam_files"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
os.makedirs(BAM_DIR, exist_ok=True)

# ============================================================
# PRIORITY SAMPLES - Direct Google Drive Links
# ============================================================
PRIORITY_FILES = {
    "VK155": "1H03qH351o_RemyhZeSf2euICP-OE4Spw",  # Bodzia female (mtDNA H1c)
    "VK157": "1qK86VzMrAA_pzf1okvkjCgW5q2_XPAZS",  # Bodzia elite male (Y-DNA I-S2077)
}

# Download priority BAM files directly from Drive
print("📥 Downloading priority BAM files from Google Drive...\n")

bam_files = []
for sample, file_id in PRIORITY_FILES.items():
    local_path = f"{BAM_DIR}/{sample}.final.bam"
    
    if os.path.exists(local_path):
        print(f"✓ {sample}.final.bam already downloaded")
    else:
        print(f"⬇️  Downloading {sample}.final.bam...")
        # Use gdown for direct Drive download
        !pip install -q gdown
        import gdown
        url = f"https://drive.google.com/uc?id={file_id}"
        gdown.download(url, local_path, quiet=False)
        print(f"✓ {sample}.final.bam downloaded")
    
    if os.path.exists(local_path):
        size_gb = os.path.getsize(local_path) / (1024**3)
        print(f"   Size: {size_gb:.2f} GB")
        bam_files.append(local_path)

print(f"\n📊 Ready to process {len(bam_files)} priority samples")
print(f"💾 Output will be saved to: {DRIVE_OUTPUT}")

In [ ]:
# Step 6: Define batch processing function
import subprocess
import time

def convert_bam_to_gedmatch(bam_path, ref_genome, dbsnp, output_dir, drive_output):
    """Convert a single BAM file to GedMatch 23andMe format."""
    sample = os.path.basename(bam_path).replace('.final.bam', '')
    start_time = time.time()
    
    print(f"\n{'='*60}")
    print(f"Processing: {sample}")
    print(f"{'='*60}")
    
    try:
        # Check if already processed
        final_output = f"{drive_output}/{sample}_23andme.txt"
        if os.path.exists(final_output):
            print(f"⏭️  Skipping {sample} - already processed")
            return {"sample": sample, "status": "skipped", "snps": 0}
        
        # Index BAM if needed
        if not os.path.exists(f"{bam_path}.bai"):
            print(f"[1/6] Indexing BAM...")
            subprocess.run(["samtools", "index", bam_path], check=True)
        else:
            print(f"[1/6] BAM index exists ✓")
        
        # Variant calling
        print(f"[2/6] Variant calling (chr 1-22)...")
        raw_vcf = f"{output_dir}/{sample}_raw.vcf.gz"
        mpileup = subprocess.Popen(
            ["bcftools", "mpileup", "-Ou", "-f", ref_genome,
             "-r", "1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22",
             "--max-depth", "250", "--min-MQ", "20", bam_path],
            stdout=subprocess.PIPE, stderr=subprocess.DEVNULL
        )
        subprocess.run(
            ["bcftools", "call", "-mv", "-Oz", "-o", raw_vcf],
            stdin=mpileup.stdout, check=True, stderr=subprocess.DEVNULL
        )
        subprocess.run(["bcftools", "index", raw_vcf], check=True)
        
        # Filter
        print(f"[3/6] Filtering (QUAL >= 20)...")
        filtered_vcf = f"{output_dir}/{sample}_filtered.vcf.gz"
        subprocess.run(
            ["bcftools", "filter", "-i", "QUAL>=20", raw_vcf, "-Oz", "-o", filtered_vcf],
            check=True
        )
        
        # Normalize
        print(f"[4/6] Normalizing...")
        normalized_vcf = f"{output_dir}/{sample}_normalized.vcf.gz"
        subprocess.run(
            ["bcftools", "norm", "-f", ref_genome, filtered_vcf, "-Oz", "-o", normalized_vcf],
            check=True, stderr=subprocess.DEVNULL
        )
        subprocess.run(["bcftools", "index", normalized_vcf], check=True)
        
        # Annotate with rsIDs
        print(f"[5/6] Annotating with rsIDs...")
        annotated_vcf = f"{output_dir}/{sample}_annotated.vcf.gz"
        subprocess.run(
            ["bcftools", "annotate", "-a", dbsnp, "-c", "ID", normalized_vcf, "-Oz", "-o", annotated_vcf],
            check=True, stderr=subprocess.DEVNULL
        )
        subprocess.run(["bcftools", "index", annotated_vcf], check=True)
        
        # Convert to 23andMe format
        print(f"[6/6] Converting to 23andMe format...")
        snps_vcf = f"{output_dir}/{sample}_snps.vcf.gz"
        subprocess.run(
            ["bcftools", "view", "-i", 'ID!="." && strlen(REF)==1 && strlen(ALT)==1',
             annotated_vcf, "-Oz", "-o", snps_vcf],
            check=True
        )
        
        plink_out = f"{output_dir}/{sample}_plink"
        subprocess.run(
            ["plink", "--vcf", snps_vcf, "--recode", "23", "--chr", "1-22", "--out", plink_out],
            check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
        
        # Create final output
        local_output = f"{output_dir}/{sample}_23andme.txt"
        with open(local_output, 'w') as out:
            out.write("# rsid\tchromosome\tposition\tgenotype\n")
            with open(f"{plink_out}.txt", 'r') as inp:
                next(inp)  # Skip header
                for line in inp:
                    out.write(line)
        
        # Count SNPs
        with open(local_output, 'r') as f:
            snp_count = sum(1 for _ in f) - 1  # Subtract header
        
        # Copy to Drive
        subprocess.run(["cp", local_output, final_output], check=True)
        
        # Cleanup intermediate files
        for f in [raw_vcf, f"{raw_vcf}.csi", filtered_vcf, normalized_vcf, f"{normalized_vcf}.csi",
                  annotated_vcf, f"{annotated_vcf}.csi", snps_vcf,
                  f"{plink_out}.txt", f"{plink_out}.log", f"{plink_out}.nosex", local_output]:
            if os.path.exists(f):
                os.remove(f)
        
        elapsed = time.time() - start_time
        print(f"\n✓ {sample} complete: {snp_count:,} SNPs in {elapsed/60:.1f} min")
        
        return {"sample": sample, "status": "success", "snps": snp_count, "time": elapsed}
        
    except Exception as e:
        print(f"\n✗ {sample} FAILED: {str(e)}")
        return {"sample": sample, "status": "failed", "error": str(e)}

print("Batch processing function defined ✓")

In [ ]:
# Step 7: Run batch processing
from datetime import datetime

print(f"\n{'#'*60}")
print(f"# BATCH PROCESSING STARTED: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"# Samples to process: {len(bam_files)}")
print(f"{'#'*60}")

results = []
total_start = time.time()

for i, bam_file in enumerate(sorted(bam_files), 1):
    print(f"\n[{i}/{len(bam_files)}] ", end="")
    result = convert_bam_to_gedmatch(
        bam_file, REF_GENOME, DBSNP, OUTPUT_DIR, DRIVE_OUTPUT
    )
    results.append(result)

total_elapsed = time.time() - total_start

print(f"\n\n{'#'*60}")
print(f"# BATCH PROCESSING COMPLETE")
print(f"# Total time: {total_elapsed/60:.1f} minutes")
print(f"{'#'*60}")

In [ ]:
# Step 8: Summary report
import pandas as pd

print("\n" + "="*70)
print("CONVERSION SUMMARY")
print("="*70)

successful = [r for r in results if r["status"] == "success"]
skipped = [r for r in results if r["status"] == "skipped"]
failed = [r for r in results if r["status"] == "failed"]

print(f"\n✓ Successful: {len(successful)}")
print(f"⏭️  Skipped:    {len(skipped)}")
print(f"✗ Failed:     {len(failed)}")

if successful:
    print(f"\n{'Sample':<12} {'SNPs':>12} {'Time (min)':>12} {'Quality':>15}")
    print("-" * 55)
    for r in sorted(successful, key=lambda x: x["snps"], reverse=True):
        quality = "Good" if r["snps"] >= 200000 else "Low (aDNA)" if r["snps"] >= 50000 else "Very Low"
        print(f"{r['sample']:<12} {r['snps']:>12,} {r['time']/60:>12.1f} {quality:>15}")
    
    total_snps = sum(r["snps"] for r in successful)
    avg_snps = total_snps / len(successful)
    print("-" * 55)
    print(f"{'Average':<12} {avg_snps:>12,.0f}")

if failed:
    print(f"\n⚠️  Failed samples:")
    for r in failed:
        print(f"  • {r['sample']}: {r.get('error', 'Unknown error')}")

print(f"\n\nOutput files saved to: {DRIVE_OUTPUT}")
print("\nTo upload to GedMatch:")
print("  1. Go to https://www.gedmatch.com")
print("  2. Log in → DNA Upload")
print("  3. Select '23andMe' format")
print(f"  4. Upload files from: GedMatch_Files/")

In [ ]:
# Step 9: List generated files
print("\nGenerated GedMatch files:")
print("-" * 50)
!ls -lh "{DRIVE_OUTPUT}"/*.txt 2>/dev/null || echo "No files found"

## Notes on Ancient DNA Quality

**Expected SNP counts for ancient DNA:**
- Modern consumer tests: 600,000 - 700,000 SNPs
- High-coverage ancient DNA (>10x): 200,000+ SNPs
- Medium-coverage ancient DNA (2-10x): 100,000 - 200,000 SNPs
- Low-coverage ancient DNA (<2x): 50,000 - 100,000 SNPs

**GedMatch compatibility:**
- Minimum ~50,000 SNPs for upload
- Matches will be fewer and shorter segments
- Most useful for population-level analysis (Admixture, Oracle)
- One-to-one comparisons may show artificially high shared DNA due to missing data

**Bodzia cemetery context:**
- Samples are ~1,000 years old
- Expected coverage: 0.5-3x
- Typical SNP yield: 50,000 - 150,000